## 练习一，找出每天不同渠道的总人数和总支出（这里的渠道有单纯的手机端，有单纯的电脑端还有两者都用）

思路：
1.先按人进行划分，找出每个人投入的手机支出和电脑端支出，打标用户最终类型

2.再进行聚合汇总

In [1]:
import pandas as pd
import numpy as np
spending=pd.read_csv("C:/Users/1/data2/pandas111.csv")

In [2]:
spending.head(10)

,member_id,date,channel,spend
0,1001,1/1/2018,mobile,100
1,1001,1/1/2018,desktop,100
2,1002,1/1/2018,mobile,100
3,1002,1/2/2018,mobile,100
4,1003,1/1/2018,desktop,100
5,1003,1/2/2018,desktop,100


In [3]:
## 先在原表基础上增加两列，代表花费
spending['mobile_spend'] = spending[spending.channel == 'mobile'].spend
spending['desktop_spend'] = spending[spending.channel == 'desktop'].spend
#新建一个表，用groupby求和
member_spend = spending.groupby(['date', 'member_id']).sum()[['mobile_spend','desktop_spend']].reset_index()


In [4]:
member_spend.head()

,date,member_id,mobile_spend,desktop_spend
0,1/1/2018,1001,100.0,100.0
1,1/1/2018,1002,100.0,0.0
2,1/1/2018,1003,0.0,100.0
3,1/2/2018,1002,100.0,0.0
4,1/2/2018,1003,0.0,100.0


In [5]:
member_spend.loc[(member_spend.mobile_spend>0) & (member_spend.desktop_spend==0),'channel'] = 'mobile'
member_spend.loc[(member_spend.mobile_spend==0) & (member_spend.desktop_spend>0), 'channel'] = 'desktop'
member_spend.loc[(member_spend.mobile_spend>0) & (member_spend.desktop_spend>0), 'channel'] = 'both'

In [6]:
tot_members = member_spend.groupby(['date', 'channel']).size().to_frame('tot_members').reset_index()
tot_spend = member_spend.groupby(['date', 'channel']).agg({'mobile_spend':sum, 'desktop_spend':sum})[['mobile_spend', 'desktop_spend']].reset_index()
tot_spend['tot_spend'] = tot_spend['mobile_spend'] + tot_spend['desktop_spend']


In [7]:
output=pd.concat([tot_members, tot_spend['tot_spend']], axis=1)
output.head()

,date,channel,tot_members,tot_spend
0,1/1/2018,both,1,200.0
1,1/1/2018,desktop,1,100.0
2,1/1/2018,mobile,1,100.0
3,1/2/2018,desktop,1,100.0
4,1/2/2018,mobile,1,100.0


##   练习二 找出不同用户的人均时间
播放记录表如下，计算每个⽤户在这个⻚⾯的次均停留时⻓，即每个⽤户相邻leave时刻-enter
时刻的均值，表 A：记录每个⽤户进⼊⼀个⻚⾯的状态和这个状态发⽣的时刻，状态只有两种：Enter 和 Leave，
每个⽤户 Enter 状态后必然存在⼀个 Leave 状态
shift(-1)练习（等同于SQL 中的lead函数）



In [8]:
lag11=pd.read_csv("C:/Users/1/data2/lag.csv")

In [9]:
lag11.head(100)

,useid,status,time
0,1,Enter,2019/4/19 3:09
1,1,Leave,2019/4/19 3:10
2,2,Enter,2019/4/19 3:11
3,3,Enter,2019/4/19 3:19
4,2,Leave,2019/4/19 3:20
5,1,Enter,2019/4/19 3:29
6,3,Leave,2019/4/19 4:22
7,1,Leave,2019/4/19 4:56


In [10]:
lag11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
useid     8 non-null int64
status    8 non-null object
time      8 non-null object
dtypes: int64(1), object(2)
memory usage: 320.0+ bytes


In [11]:
lag11['next_time'] = lag11.sort_values(['useid', 'time']).groupby('useid')['time'].shift(-1)
lag11=lag11[lag11.status=='Enter']

In [12]:
lag11.head()

,useid,status,time,next_time
0,1,Enter,2019/4/19 3:09,2019/4/19 3:10
2,2,Enter,2019/4/19 3:11,2019/4/19 3:20
3,3,Enter,2019/4/19 3:19,2019/4/19 4:22
5,1,Enter,2019/4/19 3:29,2019/4/19 4:56


In [13]:
lag11['time_cha']=(pd.to_datetime(lag11['next_time'])-pd.to_datetime(lag11['time'])).dt.total_seconds()

In [14]:
lag11.head()

,useid,status,time,next_time,time_cha
0,1,Enter,2019/4/19 3:09,2019/4/19 3:10,60.0
2,2,Enter,2019/4/19 3:11,2019/4/19 3:20,540.0
3,3,Enter,2019/4/19 3:19,2019/4/19 4:22,3780.0
5,1,Enter,2019/4/19 3:29,2019/4/19 4:56,5220.0


In [15]:
output=lag11.groupby(['useid']).agg({'time_cha':'mean'}).reset_index()
output.head()

,useid,time_cha
0,1,2640.0
1,2,540.0
2,3,3780.0


In [ ]:
##用SQL来写的话，如下：
##=====================
select userid,avg(datediff(next_time,time)) as avg_time
 from
(
select userid,status,time,lead(1,0) over(partition by userid order by time) as next_time 
from biao
)t
where status="Enter"
group by userid
;

##======================

### 关于左右表连接的练习
找2月1日上传的在2月2日浏览但没点赞的各个省份视频数
有用资料：https://www.ershicimi.com/p/2153a2ad0f9cbb8ad376bbd3b2326609



In [16]:
import pandas as pd
import numpy as np
upvote=pd.read_csv("C:/Users/1/data2/practice/upvote.csv",encoding = 'GB2312')
view=pd.read_csv("C:/Users/1/data2/practice/view.csv",encoding = 'GB2312')
photo=pd.read_csv("C:/Users/1/data2/practice/photo.csv",encoding = 'GB2312')

In [17]:
photo.head(10)

,author_id,photo_id,upload_time,province
0,1,4000,2020/2/1 2:00,河北
1,2,4001,2020/2/1 2:01,广东
2,3,4002,2020/2/2 2:02,山东
3,3,4003,2020/2/1 2:03,山东
4,5,4004,2020/2/3 2:04,辽宁
5,4,4005,2020/2/5 2:05,新疆
6,13,4300,2020/2/1 2:01,广东
7,15,4004,2020/2/1 2:01,新疆
8,15,4007,2020/2/1 2:01,山东
9,18,4009,2020/2/1 2:05,新疆


In [18]:
view.head()

,user_id,photo_id,date,province
0,11,5000,2020/2/1,广东
1,13,4300,2020/2/2,黑龙江
2,14,4200,2020/2/1,辽宁
3,11,4300,2020/2/1,广东
4,12,4001,2020/2/1,广东


In [19]:
upvote.head()

,user_id,photo_id,date,province
0,11,5001,2020/2/1,广东
1,12,4300,2020/2/2,广东
2,13,4200,2020/2/1,黑龙江
3,14,4300,2020/2/1,辽宁
4,12,4001,2020/2/1,广东


In [20]:
##将表中的时间转换一下
photo['upload_time'] = pd.to_datetime(photo['upload_time'])
view['date'] = pd.to_datetime(view['date'])
upvote['date'] = pd.to_datetime(upvote['date'])

#且先把时间限定清楚，就不用后面调整了
someday = pd.to_datetime('2020-02-01')
someday1=pd.to_datetime('2020-02-02')
photo=photo[photo['upload_time'].dt.date==someday]
view=view[view['date']==someday1]
upvote=upvote[upvote['date']==someday1]

In [21]:
#参照这个链接：https://stackoverflow.com/questions/49487263/pandas-left-join-where-right-is-null-on-multiple-columns
zichaxun=pd.merge(view,upvote,indicator='i',left_on=['user_id','photo_id'],right_on=['user_id','photo_id'],how='left').query('i == "left_only"').drop('i', 1)
zichaxun


,user_id,photo_id,date_x,province_x,date_y,province_y
0,13,4300,2020-02-02,黑龙江,NaT,NaN
3,16,4004,2020-02-02,云南,NaT,NaN
4,17,4004,2020-02-02,云南,NaT,NaN
5,18,4004,2020-02-02,云南,NaT,NaN
6,20,4007,2020-02-02,成都,NaT,NaN
7,21,4009,2020-02-02,北京,NaT,NaN


In [22]:
erceng=pd.merge(zichaxun,photo,on=['photo_id'],how='inner')
erceng

,user_id,photo_id,date_x,province_x,date_y,province_y,author_id,upload_time,province
0,13,4300,2020-02-02,黑龙江,NaT,NaN,13,2020-02-01 02:01:00,广东
1,16,4004,2020-02-02,云南,NaT,NaN,15,2020-02-01 02:01:00,新疆
2,17,4004,2020-02-02,云南,NaT,NaN,15,2020-02-01 02:01:00,新疆
3,18,4004,2020-02-02,云南,NaT,NaN,15,2020-02-01 02:01:00,新疆
4,20,4007,2020-02-02,成都,NaT,NaN,15,2020-02-01 02:01:00,山东
5,21,4009,2020-02-02,北京,NaT,NaN,18,2020-02-01 02:05:00,新疆


In [23]:
erceng.groupby(['province','photo_id']).size().to_frame('各级').reset_index()

,province,photo_id,各级
0,山东,4007,1
1,广东,4300,1
2,新疆,4004,3
3,新疆,4009,1


In [24]:
##这里的first就是用的row_number 语句；sort_values就是sql里面的排序
erceng['rank'] = erceng.groupby('province')['photo_id'].rank(method = 'first', ascending = False)
erceng.sort_values(['province', 'photo_id'])

,user_id,photo_id,date_x,province_x,date_y,province_y,author_id,upload_time,province,rank
4,20,4007,2020-02-02,成都,NaT,NaN,15,2020-02-01 02:01:00,山东,1.0
0,13,4300,2020-02-02,黑龙江,NaT,NaN,13,2020-02-01 02:01:00,广东,1.0
1,16,4004,2020-02-02,云南,NaT,NaN,15,2020-02-01 02:01:00,新疆,2.0
2,17,4004,2020-02-02,云南,NaT,NaN,15,2020-02-01 02:01:00,新疆,3.0
3,18,4004,2020-02-02,云南,NaT,NaN,15,2020-02-01 02:01:00,新疆,4.0
5,21,4009,2020-02-02,北京,NaT,NaN,18,2020-02-01 02:05:00,新疆,1.0


In [25]:
erceng.groupby('province')['photo_id'].nunique().reset_index()

,province,photo_id
0,山东,1
1,广东,1
2,新疆,2
